In [4]:
import glob
from tqdm import tqdm
from datasets import load_dataset
import json
from sentence_transformers import SentenceTransformer, losses, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
import gzip
import os
from sentence_transformers import InputExample
from datasets import Dataset
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def get_data(path, data_type):
    cnt = 0
    data_dict = {
        'anchor': [],
        'positive': []
    }
    results_path = glob.glob(path + '/*.json')
    for sample_result_path in tqdm(results_path):
        turn_num = int(sample_result_path.split('/')[-1].split('.')[0].split('_')[-1])
        sample_result = json.load(open(sample_result_path))
        target_item_title = sample_result['rec'][0]
        dialog_dict = sample_result['simulator_dialog']['context']
        # assert (len(dialog_dict) - turn_num) % 2 == 0.0, f"dialog_dict: {len(dialog_dict)}, turn_num: {turn_num}"
        if (len(dialog_dict) - turn_num) % 2 != 0.0: continue
        iter_num = (len(dialog_dict) - turn_num) // 2
        gt_item_list = sample_result['rec']
        print(f'iter_num: {iter_num}, turn_num: {turn_num}, len(dialog_dict): {len(dialog_dict)}, sample_result_path: {sample_result_path}')
        for gt_item in gt_item_list:
            passage = gt_item
            if data_type == 'total':
                for idx in range(1, iter_num+1):
                    if idx == 0:
                        query_dialog = dialog_dict[:]
                    else:
                        query_dialog = dialog_dict[: -2*idx]
                    query = ""
                    for context in query_dialog:
                        query += f"{context['role']}: {context['content']} "
                    data_dict['anchor'].append(query)
                    data_dict['positive'].append(passage)
                    print(f'idx: {idx}, query: {query}, passage: {passage}')
            else:
                query_dialog = dialog_dict[: -2*iter_num]
                query = ""
                for context in query_dialog:
                    query += f"{context['role']}: {context['content']} "
                data_dict['anchor'].append(query)
                data_dict['positive'].append(passage)
            
    data = Dataset.from_dict(data_dict)
    return data

data_type = 'total'
# embedding_model = "nomic-ai/nomic-embed-text-v1"
embedding_model = "nomic-ai/nomic-embed-text-v1"
rec_model = "Llama-3.2-1B-Instruct"
# adapter_model = "Llama-3.2-1B-Instruct-DPO-tuned-new-reward"
# adapter_model = "Llama-3.2-1B-Instruct-DPO-tuned-nomic-no-tuned"
adapter_model = None
topK= 10
history = "full"
split = "train"
user_model = "Llama-3.1-8B-Instruct"
prompt_ver = 'v2'
turn_num = 2
# Train the model
num_epochs = 7
lr = 5e-5

# temp = embedding_model
# embedding_model = "Snowflake/snowflake-arctic-embed-s"
if adapter_model == None:
    train_dir = f'/home/shchoi/iEvaLM-CRS/save_{turn_num}/{prompt_ver}/user_{user_model}/emb_{embedding_model}/openmodel_{rec_model}_top{topK}_{history}_history/opendialkg_eval/full_non_repeated/train'
    eval_dir = f'/home/shchoi/iEvaLM-CRS/save_{turn_num}/{prompt_ver}/user_{user_model}/emb_{embedding_model}/openmodel_{rec_model}_top{topK}_{history}_history/opendialkg_eval/full_non_repeated/valid' 
else:
    train_dir = f'/home/shchoi/iEvaLM-CRS/save_{turn_num}/{prompt_ver}/user_{user_model}/emb_{embedding_model}/openmodel_{rec_model}_adapter_{adapter_model}_top{topK}_{history}_history/opendialkg_eval/full_non_repeated/train'
    eval_dir = f'/home/shchoi/iEvaLM-CRS/save_{turn_num}/{prompt_ver}/user_{user_model}/emb_{embedding_model}/openmodel_{rec_model}_adapter_{adapter_model}_top{topK}_{history}_history/opendialkg_eval/full_non_repeated/test' 
# embedding_model = temp
print(train_dir)
training_data = get_data(train_dir, data_type)
eval_data = get_data(eval_dir, data_type)
print(train_dir)
# print(gt_dir)

/home/shchoi/iEvaLM-CRS/save_2/v2/user_Llama-3.1-8B-Instruct/emb_nomic-ai/nomic-embed-text-v1/openmodel_Llama-3.2-1B-Instruct_top10_full_history/opendialkg_eval/full_non_repeated/train


  0%|          | 10/3560 [00:00<00:00, 3688.92it/s]


iter_num: 1, turn_num: 3, len(dialog_dict): 5, sample_result_path: /home/shchoi/iEvaLM-CRS/save_2/v2/user_Llama-3.1-8B-Instruct/emb_nomic-ai/nomic-embed-text-v1/openmodel_Llama-3.2-1B-Instruct_top10_full_history/opendialkg_eval/full_non_repeated/train/5155_3.json
idx: 1, query: user: Can you tell me the genre of Kathleen Grissom's books? assistant: Most of her book are Historical Fiction genre like The kitchen House. Do you like Historical Fiction? user: It depends. Are they about wars? , passage: The Kitchen House
iter_num: 2, turn_num: 3, len(dialog_dict): 7, sample_result_path: /home/shchoi/iEvaLM-CRS/save_2/v2/user_Llama-3.1-8B-Instruct/emb_nomic-ai/nomic-embed-text-v1/openmodel_Llama-3.2-1B-Instruct_top10_full_history/opendialkg_eval/full_non_repeated/train/1512_3.json
idx: 1, query: user: Do you like Aaron Rodgers? assistant: Are you referring to the football player for the Green Bay Packers? user: Yes, I am. Isn't he a great player? assistant: Aaron Rodgers is indeed an exceptio

  1%|▏         | 10/757 [00:00<00:00, 3655.49it/s]

iter_num: 1, turn_num: 1, len(dialog_dict): 3, sample_result_path: /home/shchoi/iEvaLM-CRS/save_2/v2/user_Llama-3.1-8B-Instruct/emb_nomic-ai/nomic-embed-text-v1/openmodel_Llama-3.2-1B-Instruct_top10_full_history/opendialkg_eval/full_non_repeated/valid/2763_1.json
idx: 1, query: user: I have been a fan of Zack Ward since back when he was a kid in Christmas Story. Can you recommend a more recent movie he starred in? , passage: Restoration
iter_num: 1, turn_num: 1, len(dialog_dict): 3, sample_result_path: /home/shchoi/iEvaLM-CRS/save_2/v2/user_Llama-3.1-8B-Instruct/emb_nomic-ai/nomic-embed-text-v1/openmodel_Llama-3.2-1B-Instruct_top10_full_history/opendialkg_eval/full_non_repeated/valid/4173_1.json
idx: 1, query: user: Do you have any book titles written by Donna Tartt? , passage: The Secret History
iter_num: 2, turn_num: 1, len(dialog_dict): 5, sample_result_path: /home/shchoi/iEvaLM-CRS/save_2/v2/user_Llama-3.1-8B-Instruct/emb_nomic-ai/nomic-embed-text-v1/openmodel_Llama-3.2-1B-Instruct

In [ ]:
training_data

In [ ]:
from torch.utils.data import DataLoader
import wandb

from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction, InformationRetrievalEvaluator
from sentence_transformers.training_args import BatchSamplers
# embedding_model = "nomic-ai/nomic-embed-text-v1-already-tuned-lr_te-5"
# embedding_model = "nomic-ai/nomic-embed-text-v1-after-tuned"
model = SentenceTransformer(embedding_model, cache_folder = "/data1/shchoi/LLM_ckp/hub", device="cuda:0", trust_remote_code=True)
# model = SentenceTransformer("/home/shchoi/iEvaLM-CRS/experiment_code/output/total/user_Llama-3.1-8B-Instruct/emb_nomic-ai/nomic-embed-text-v1/openmodel_Llama-3.2-1B-Instruct_top10_full_history/opendialkg_eval/full_non_repeated/train/checkpoint-65", cache_folder = "/data1/shchoi/LLM_ckp/hub", device="cuda:0", trust_remote_code=True)
embedding_model = "nomic-ai/nomic-embed-text-v1-after-tuned"
# Initialize loss
train_loss = losses.MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=f'output/{data_type}/user_{user_model}/emb_{embedding_model}/openmodel_{rec_model}_top{topK}_{history}_history/opendialkg_eval/full_non_repeated/{split}',
    seed = 42,
    # Optional training parameters:
    learning_rate=lr,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=4, 
    warmup_ratio=0.1,
    fp16=True,  # Set to False if GPU can't handle FP16
    bf16=False,  # Set to True if GPU supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=5,
    save_strategy="steps",
    save_steps=5,
    save_total_limit=5,
    logging_steps=5,
    eval_on_start=True,
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=training_data,
    eval_dataset=eval_data,
    loss=train_loss,
    # evaluator=evaluator,
)

trainer.train()
trainer.save_model()

In [5]:
trainer.save_model()